## 12.1 Dagger2 - Wstrzykiwanie zależności - Podstawy

Aplikacja posłuży nam do zapoznania się z ideą **dependency injection**, czyli wstrzykiwaniem zależności. Wykorzystamy tylko pojedynczą aktywność z polem `TextView` (bez fragmentów i nawigacji).

Rozpoczniemy od dodania odpowiednich zależności do projektu. Zależności możemy skopiować ze strony projektu na [github](https://github.com/google/dagger).

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-kapt'
}
...
dependencies {

    implementation 'com.google.dagger:dagger:2.44'
    annotationProcessor 'com.google.dagger:dagger-compiler:2.44'

    implementation 'com.google.dagger:dagger-android:2.44'
    implementation 'com.google.dagger:dagger-android-support:2.44'
    annotationProcessor 'com.google.dagger:dagger-android-processor:2.44'

    kapt 'com.google.dagger:dagger-android-processor:2.44'
    kapt 'com.google.dagger:dagger-compiler:2.44'
}

Napiszmy kilka klas reprezentujących komputrer oraz jego składowe.

In [ ]:
class Computer {}
class Case {}
class CPU {}
class GPU {}
class Motherboard {}
class PowerSupply {}

Klasa `Computer` nie powinna odpowiadać za utworzenie zależności, więc przekazemy je do konstruktora głównego. Dodamy również jedną metodę.

In [ ]:
class Computer (
    private val case: Case,
    private val gpu: GPU,
    private val cpu: CPU,
    private val motherboard: Motherboard,
    private val powerSupply: PowerSupply
        ) {
    fun work(): String{
        return "working"
    }
}

Przejdźmy do głównej aktynwości i dodajmy zmienną `Computer` oraz ustawmy text pola `TextView` w metodzie `onCreate`.

In [ ]:
class MainActivity : AppCompatActivity() {

    private lateinit var computer: Computer

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val textView = findViewById<TextView>(R.id.textview)
        textView.text = computer.work()
    }
}

Musimy jeszcze zainicjować obiekt `Computer`. Aby to zrobić musielibyśmy utworzyć szereg innych obiektów.

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_main)
    
//     val case = Case()
//     val gpu = GPU()
//     val cpu = CPU()
//     val motherboard = Motherboard()
//     val powerSupply = PowerSupply()
    
//     computer = Computer(case, gpu, cpu, motherboard, powerSupply)

    val textView = findViewById<TextView>(R.id.textview)
    textView.text = computer.work()
}

Ponieważ nie chcemy tego robić manualnie za każdym razem, wykorzystamy bibliotekę `Dagger2`.

Utwórzmy nowy interfejs o nazwie `computerComponent` - tutaj zostanie utworzony `DAG` ([Skierowany graf acykliczny](https://pl.wikipedia.org/wiki/Skierowany_graf_acykliczny)) w którym zostaną zawarte wszystkie informacje o zależnościach klasy `Computer` (od czego zależy klasa `Computer` oraz kolejność w jakich wszystkie zależności muszą zostać utworzone).

`@Component` tworzy i przechowuje obiekt, następnie dostarcza go do odbiorcy - często nazywany *injector*

In [ ]:
@Component
interface ComputerComponent {
}

Zdefiniujmy jedną metodę `getComputer`

In [ ]:
@Component
interface ComputerComponent {
    fun getComputer(): Computer
}

`Dagger2` w czasi kompilacji zaimplementuje ten interfejs i doda automatycznie niezbędny kod. Kolejnym krokiem będzie odpowiednie oznaczenie wszystkich konstruktorów (**wstrzykiwanie przez konstruktor**) które musza zostać wykorzystane przy tworzeniu instancji klasy `Computer`.

In [ ]:
class Computer @Inject constructor (
    private val case: Case,
    private val gpu: GPU,
    private val cpu: CPU,
    private val motherboard: Motherboard,
    private val powerSupply: PowerSupply
        ) {
    fun work(): String{
        return "working"
    }
}

Aby użyć adnotacji `@Inject` musimy wykorzystać słowo kluczowe `constructor`. `Dagger2` musi jeszcze posiadać informację o sposobie zainicjowania wszystkich zależności.

In [ ]:
class Case @Inject constructor()
class CPU @Inject constructor()
class GPU @Inject constructor()
class Motherboard @Inject constructor()
class PowerSupply @Inject constructor()

W klasie `MainActivity` nie możemy zastosować adnotacji `@Inject` na konstruktorze - instancję klasy `Computer` otrzymamy dzięki interfejsowi `ComputerComponent`. Ponieważ wykorzystaliśmy adnotację `@Component` mamy dostępną klasę `DaggerComputerComponent` wygenerowaną automatycznie, posiada ona metoda `create` dzięki której możemy stworzyć obiekt o typie interfejsu `ComputerComponent`

In [ ]:
val component = DaggerComputerComponent.create()

Następnie skorzystamy z metody `getComputer` zdefiniowanej w interfejsie `ComputerComponent`

In [ ]:
computer = component.getComputer()

Pełny kod `MainActivity`

In [ ]:
class MainActivity : AppCompatActivity() {

    private lateinit var computer: Computer

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val component = DaggerComputerComponent.create()
        computer = component.getComputer()
        val textView = findViewById<TextView>(R.id.textview)
        textView.text = computer.work()
    }
}